In [1]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile("110m_cultural_all.xlsx")

# The target columns to check in each sheet
target_cols = ['GU_A3', 'SU_A3', 'BRK_A3', 'ADM0_TLC']

# Dictionary to store the matching columns for each sheet
sheet_info = {}

# Loop over each sheet and identify which target columns are present
for sheet in xls.sheet_names:
    df_sheet = pd.read_excel(xls, sheet_name=sheet)
    cols_found = [col for col in df_sheet.columns if col in target_cols]
    sheet_info[sheet] = cols_found

# Create a DataFrame summarizing the results
summary_df = pd.DataFrame(list(sheet_info.items()), columns=["Sheet", "Matching Columns"])
print(summary_df)

                              Sheet                  Matching Columns
0         ne_110m_admin_0_countries  [GU_A3, SU_A3, BRK_A3, ADM0_TLC]
1   ne_110m_admin_0_countries_lakes  [GU_A3, SU_A3, BRK_A3, ADM0_TLC]
2          ne_110m_populated_places                                []
3   ne_110m_admin_1_states_province                                []
4        ne_110m_admin_0_scale_rank                                []
5   ne_110m_admin_0_boundary_lines_                          [BRK_A3]
6   ne_110m_populated_places_simple                                []
7   ne_110m_admin_0_pacific_groupin                                []
8         ne_110m_admin_0_map_units  [GU_A3, SU_A3, BRK_A3, ADM0_TLC]
9       ne_110m_admin_0_sovereignty  [GU_A3, SU_A3, BRK_A3, ADM0_TLC]
10   ne_110m_admin_0_tiny_countries  [GU_A3, SU_A3, BRK_A3, ADM0_TLC]


In [2]:
import pandas as pd
import re
from fuzzywuzzy import process
import pycountry
import pycountry_convert as pc

# Updated cleaning function:
def clean_country_name(name):
    if pd.isna(name):
        return ""
    if not isinstance(name, str):
        name = str(name)
    name = name.lower()
    # Special handling: if "french polynesia" appears anywhere, return exactly "french polynesia"
    if "french polynesia" in name:
        return "french polynesia"
    # Remove content within parentheses and trailing ", france" if present
    name = re.sub(r"\(.*?\)", "", name)
    name = re.sub(r",\s*france$", "", name)
    # Normalize whitespace and trim
    name = re.sub(r"\s+", " ", name).strip()
    # Fix common encoding issues
    name = name.replace("c√¥te", "côte")
    name = name.replace("cura√ßao", "curaçao")
    name = name.replace("gal√°pagos", "galápagos")
    name = name.replace("r√©union", "réunion")
    name = name.replace("s√£o", "são")
    name = name.replace("t√ºrkiye", "türkiye")
    return name

# Load your Excel file as DataFrame 'df'
df = pd.read_excel("yellow_fever_malaria_data_cleaned_yfcat_tableau.xlsx")

# Create a cleaned country column based on 'Country_Tableau'
df["Country_clean"] = df["Country_Tableau"].apply(clean_country_name)

# Manual mapping for problematic names (keys are cleaned names).
# For those that should not match, we map them to None.
manual_mapping = {
    "american samoa": "american samoa",
    "anguilla": "anguilla",
    "aruba": "aruba",
    "bonaire": "bonaire",
    "cayman islands": "cayman islands",
    "christmas island": "christmas island",
    "cocos (keeling) islands": None,      # Force no match
    "easter island": None,                # Force no match
    "curaçao, netherlands": "curaçao",
    "french polynesia, including the society islands [bora-bora, moorea & tahiti]; marquesas islands": "french polynesia",
    "gibraltar (u.k.)": "gibraltar",
    "guadeloupe (including marie-galante, la désirade & îles des saintes)": "guadeloupe",
    "macau": "macau (china)",              # Map Macau correctly
    "macau (china)": "macau (china)",
    "madeira islands": "madeira islands (portugal)",  # Map Madeira Islands correctly
    "martinique": "martinique",
    "mayotte (france)": "mayotte",
    "montserrat, united kingdom": "montserrat",
    "netherlands antilles": "netherlands antilles",  # Explicit mapping
    "netherlands antilles (bonaire, curaçao, saba, st. eustasius, and st. maarten)": "netherlands antilles",
    "niue (new zealand)": "niue",
    "norfolk island (australia)": "norfolk island",
    "réunion (france)": "réunion",
    "saba, netherlands": "saba",
    "saint barthelemy, france": "saint barthelemy",
    "saint helena, united kingdom": "saint helena",
    "saint martin, france": "saint martin",
    "sint eustatius, netherlands": "sint eustatius",
    "sint maarten, netherlands": "sint maarten",
    "tokelau (new zealand)": "tokelau",
    "virgin islands, british": "virgin islands, british",
    "virgin islands, u.s.": "virgin islands, u.s.",
    "wake island, u.s.": "wake island, u.s."
}

# Build candidate mapping from the Excel file containing geographic codes.
candidate_map = {}
xls = pd.ExcelFile("110m_cultural_all.xlsx")
for sheet in xls.sheet_names:
    df_sheet = pd.read_excel(xls, sheet_name=sheet)
    # Identify columns that contain "NAME" (case-insensitive)
    name_columns = [col for col in df_sheet.columns if "NAME" in col.upper()]
    for col in name_columns:
        for idx, row in df_sheet.iterrows():
            candidate = row[col]
            if pd.isna(candidate):
                continue
            candidate_clean = clean_country_name(candidate)
            # Correct false detection for Saint Martin if detected as "saint-marin"
            if candidate_clean == "saint-marin":
                candidate_clean = "saint martin"
            candidate_info = {
                "GU_A3": row["GU_A3"] if "GU_A3" in df_sheet.columns else None,
                "SU_A3": row["SU_A3"] if "SU_A3" in df_sheet.columns else None,
                "BRK_A3": row["BRK_A3"] if "BRK_A3" in df_sheet.columns else None,
                "ADM0_TLC": row["ADM0_TLC"] if "ADM0_TLC" in df_sheet.columns else None,
                "SOV_A3": row["SOV_A3"] if "SOV_A3" in df_sheet.columns else None,
                "ADM0_A3": row["ADM0_A3"] if "ADM0_A3" in df_sheet.columns else None,
                "ISO_A2": row["ISO_A2"] if "ISO_A2" in df_sheet.columns else None,
                "SUBUNIT": row["SUBUNIT"] if "SUBUNIT" in df_sheet.columns else None,
                "SUBREGION": row["SUBREGION"] if "SUBREGION" in df_sheet.columns else None,
                "CONTINENT": row["CONTINENT"] if "CONTINENT" in df_sheet.columns else None,
                "sheet": sheet,
                "source_column": col,
                "original_candidate": candidate
            }
            # Update mapping: if candidate exists, update missing values.
            if candidate_clean in candidate_map:
                for key in ["SUBUNIT", "SUBREGION", "CONTINENT", "ISO_A2"]:
                    if candidate_map[candidate_clean][key] is None and candidate_info[key] is not None:
                        candidate_map[candidate_clean][key] = candidate_info[key]
            else:
                candidate_map[candidate_clean] = candidate_info

# Set fuzzy matching threshold.
threshold = 70

# Prepare lists for storing matching results.
BRK_A3_list = []
SOV_A3_list = []
ADM0_A3_list = []
ISO_A2_list = []
SUBUNIT_list = []
SUBREGION_list = []
CONTINENT_list = []
match_score_list = []
match_sheet_list = []
match_source_list = []
match_candidate_list = []
pycountry_used = []  # Flag if pycountry fallback was used

print("Matching results per row:")

# Iterate over each row in df using the cleaned 'Country_clean' column.
for idx, row in df.iterrows():
    country_clean = row["Country_clean"]
    original_score = None
    
    # Check if the country is explicitly in manual mapping.
    if country_clean in manual_mapping:
        best_candidate_key = manual_mapping[country_clean]
        if best_candidate_key is None:
            # Force no match for these cases.
            candidate_info = {}
            score = 0
        else:
            candidate_info = candidate_map.get(best_candidate_key, {})
            score = 100  # Perfect match assumed.
            best_match = best_candidate_key
    else:
        best_match, score = process.extractOne(country_clean, list(candidate_map.keys()))
        candidate_info = candidate_map.get(best_match, {})
    
    original_score = score

    # If candidate_info is missing, set score to 0.
    if not candidate_info.get("original_candidate"):
        score = 0

    # Reject match if the candidate is exactly "island".
    if candidate_info.get("original_candidate", "").strip().lower() == "island":
        score = 0

    # If fuzzy score is ≤ 90, try pycountry (with pycountry_convert for continent) as a fallback.
    used_pycountry = False
    if score <= 90:
        try:
            py_results = pycountry.countries.search_fuzzy(country_clean)
            if py_results:
                py_country = py_results[0]
                # Use pycountry_convert to get continent information
                try:
                    alpha2 = py_country.alpha_2  # Needed for continent conversion
                    continent_code = pc.country_alpha2_to_continent_code(alpha2)
                    continent_mapping = {
                        "AF": "Africa",
                        "AN": "Antarctica",
                        "AS": "Asia",
                        "EU": "Europe",
                        "NA": "North America",
                        "OC": "Oceania",
                        "SA": "South America"
                    }
                    continent_name = continent_mapping.get(continent_code, None)
                except Exception as e:
                    continent_name = None
                candidate_info = {
                    "GU_A3": None,
                    "SU_A3": None,
                    "BRK_A3": py_country.alpha_3,  # Using BRK_A3 from pycountry
                    "ADM0_TLC": None,
                    "SOV_A3": None,
                    "ADM0_A3": None,
                    "ISO_A2": py_country.alpha_2,   # Retrieve ISO_A2 here
                    "SUBUNIT": None,
                    "SUBREGION": None,
                    "CONTINENT": continent_name,
                    "sheet": "pycountry",
                    "source_column": "pycountry",
                    "original_candidate": py_country.name
                }
                score = 100
                used_pycountry = True
        except Exception as e:
            pass

    print(f"Excel Country: '{row['Country_Tableau']}' (clean: '{country_clean}')")
    print(f"  -> Best match candidate: '{candidate_info.get('original_candidate')}'")
    print(f"     from sheet: '{candidate_info.get('sheet')}', column: '{candidate_info.get('source_column')}'")
    print(f"  -> Initial fuzzy score: {original_score} | Final score: {score}")
    print(f"  -> Codes: BRK_A3={candidate_info.get('BRK_A3')}, SOV_A3={candidate_info.get('SOV_A3')}, ADM0_A3={candidate_info.get('ADM0_A3')}")
    print(f"  -> Additional: ISO_A2={candidate_info.get('ISO_A2')}, SUBUNIT={candidate_info.get('SUBUNIT')}, SUBREGION={candidate_info.get('SUBREGION')}, CONTINENT={candidate_info.get('CONTINENT')}")
    if used_pycountry:
        print("  -> Fallback using pycountry (and pycountry_convert) was used.\n")
    else:
        print("\n")
    
    # Accept the match if final score meets the threshold; otherwise, set codes to None.
    if score < threshold:
        BRK_A3_list.append(None)
        SOV_A3_list.append(None)
        ADM0_A3_list.append(None)
        ISO_A2_list.append(None)
        SUBUNIT_list.append(None)
        SUBREGION_list.append(None)
        CONTINENT_list.append(None)
    else:
        BRK_A3_list.append(candidate_info.get("BRK_A3"))
        SOV_A3_list.append(candidate_info.get("SOV_A3"))
        ADM0_A3_list.append(candidate_info.get("ADM0_A3"))
        ISO_A2_list.append(candidate_info.get("ISO_A2"))
        SUBUNIT_list.append(candidate_info.get("SUBUNIT"))
        SUBREGION_list.append(candidate_info.get("SUBREGION"))
        CONTINENT_list.append(candidate_info.get("CONTINENT"))
    
    match_score_list.append(score)
    match_sheet_list.append(candidate_info.get("sheet"))
    match_source_list.append(candidate_info.get("source_column"))
    match_candidate_list.append(candidate_info.get("original_candidate"))
    pycountry_used.append(used_pycountry)

# Append matching results to the DataFrame (without '_best' in column names).
df["BRK_A3"] = BRK_A3_list
df["SOV_A3"] = SOV_A3_list
df["ADM0_A3"] = ADM0_A3_list
df["ISO_A2"] = ISO_A2_list
df["SUBUNIT"] = SUBUNIT_list
df["SUBREGION"] = SUBREGION_list
df["CONTINENT"] = CONTINENT_list
df["match_score"] = match_score_list
df["match_sheet"] = match_sheet_list
df["match_source"] = match_source_list
df["match_candidate"] = match_candidate_list
df["pycountry_used"] = pycountry_used

print("\nFinal merged DataFrame head:")
print(df.head())

Matching results per row:
Excel Country: 'Afghanistan' (clean: 'afghanistan')
  -> Best match candidate: 'Afghanistan'
     from sheet: 'ne_110m_admin_0_countries', column: 'NAME'
  -> Initial fuzzy score: 100 | Final score: 100
  -> Codes: BRK_A3=AFG, SOV_A3=AFG, ADM0_A3=AFG
  -> Additional: ISO_A2=AF, SUBUNIT=Afghanistan, SUBREGION=Southern Asia, CONTINENT=Asia


Excel Country: 'Albania' (clean: 'albania')
  -> Best match candidate: 'Albania'
     from sheet: 'ne_110m_admin_0_countries', column: 'NAME'
  -> Initial fuzzy score: 100 | Final score: 100
  -> Codes: BRK_A3=ALB, SOV_A3=ALB, ADM0_A3=ALB
  -> Additional: ISO_A2=AL, SUBUNIT=Albania, SUBREGION=Southern Europe, CONTINENT=Europe


Excel Country: 'Algeria' (clean: 'algeria')
  -> Best match candidate: 'Algeria'
     from sheet: 'ne_110m_admin_0_countries', column: 'NAME'
  -> Initial fuzzy score: 100 | Final score: 100
  -> Codes: BRK_A3=DZA, SOV_A3=DZA, ADM0_A3=DZA
  -> Additional: ISO_A2=DZ, SUBUNIT=Algeria, SUBREGION=Northern

In [3]:
# List every Country that originally had a fuzzy score ≤ 90 (before pycountry fallback).
low_score_countries = df[df["match_score"] < 100]["Country_Tableau"].unique()
print("\nCountries with original fuzzy score ≤ 100 (subject to pycountry fallback):")
print(low_score_countries)


Countries with original fuzzy score ≤ 100 (subject to pycountry fallback):
['Cocos (Keeling) Islands' 'Republic of Congo'
 'Democratic Republic of Congo' 'Easter Island' 'Macau' 'Madeira Islands'
 'Netherlands Antilles']


In [10]:
# Manual mappings for CONTINENT using normalized (lower-case) country names.
manual_continent = {
    'wake island': 'Oceania',
    'american samoa  (us)': 'Oceania',
    'andorra': 'Europe',
    'anguilla (u.k.)': 'North America',
    'antigua and barbuda': 'North America',
    'aruba': 'North America',
    'bonaire': 'North America',
    'cape verde': 'Africa',
    'cayman islands (u.k.)': 'North America',
    'christmas island': 'Oceania',
    'curaçao': 'North America',
    'dominica': 'North America',
    'gibraltar': 'Europe',
    'grenada': 'North America',
    'guadeloupe': 'North America',
    'hong kong (china)': 'Asia',
    'liechtenstein': 'Europe',
    'martinique': 'North America',
    'mayotte': 'Africa',
    'monaco': 'Europe',
    'montserrat': 'North America',
    'niue': 'Oceania',
    'norfolk island': 'Oceania',
    'réunion (france)': 'Africa',
    'saba': 'North America',
    'saint barthelemy': 'North America',
    'saint kitts and nevis': 'North America',
    'saint lucia': 'North America',
    'saint martin': 'North America',
    'saint vincent and the grenadines': 'North America',
    'san marino': 'Europe',
    'seychelles': 'Africa',
    'sint eustatius': 'North America',
    'sint maarten (dutch part)': 'North America',
    'tokelau': 'Oceania',
    'virgin islands, british': 'North America',
    'virgin islands, u.s.': 'North America'
}

# Manual mappings for SUBREGION using normalized (lower-case) country names.
manual_subregion = {
    'wake island': 'Micronesia',
    'american samoa  (us)': 'Polynesia',
    'american samoa': 'Polynesia', # American Samoa variants
    'andorra': 'Southern Europe',
    'anguilla (u.k.)': 'Caribbean',
    'antigua and barbuda': 'Caribbean',
    'aruba': 'Caribbean',
    'bonaire': 'Caribbean',
    'cape verde': 'Western Africa',
    'cayman islands (u.k.)': 'Caribbean',
    'christmas island': 'Oceania',
    'curaçao': 'Caribbean',
    'dominica': 'Caribbean',
    'gibraltar': 'Southern Europe',
    'grenada': 'Caribbean',
    'guadeloupe': 'Caribbean',
    'hong kong (china)': 'Eastern Asia',
    'liechtenstein': 'Western Europe',
    'martinique': 'Caribbean',
    'mayotte': 'Eastern Africa',
    'monaco': 'Western Europe',
    'montserrat': 'Caribbean',
    'niue': 'Polynesia',
    'norfolk island': 'Melanesia',
    'réunion (france)': 'Eastern Africa',
    'saba': 'Caribbean',
    'saint barthelemy': 'Caribbean',
    'saint kitts and nevis': 'Caribbean',
    'saint lucia': 'Caribbean',
    'saint martin': 'Caribbean',
    'saint vincent and the grenadines': 'Caribbean',
    'san marino': 'Southern Europe',
    'seychelles': 'Eastern Africa',
    'sint eustatius': 'Caribbean',
    'sint maarten (dutch part)': 'Caribbean',
    'tokelau': 'Polynesia',
    'virgin islands, british': 'Caribbean',
    'virgin islands, u.s.': 'Caribbean',
    # Additional mappings as requested:
    'cocos (keeling) islands': 'South-Eastern Asia',
    'macau': 'Eastern Asia'
}

# Helper functions for manual mapping using lower-case keys.
def get_manual_continent(country_name):
    if not country_name or not isinstance(country_name, str):
        return None
    return manual_continent.get(country_name.strip().lower())

def get_manual_subregion(country_name):
    if not country_name or not isinstance(country_name, str):
        return None
    return manual_subregion.get(country_name.strip().lower())

# Function to get continent from country name using pycountry_convert.
def get_continent_from_name(country):
    try:
        country_code = pc.country_name_to_country_alpha2(country)
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except Exception:
        return None

# Function to convert an alpha-3 country code to an alpha-2 code.
def convert_alpha3_to_alpha2(alpha3_code):
    try:
        country = pycountry.countries.get(alpha_3=alpha3_code)
        if country:
            return country.alpha_2
    except Exception:
        pass
    return None

# Helper function to get country name from ISO alpha-2 code.
def get_country_name_from_iso2(iso2):
    try:
        country = pycountry.countries.get(alpha_2=iso2)
        if country:
            return country.name
    except Exception:
        pass
    return None

# Updated function to determine continent from a row.
def get_continent_from_row(row):
    # 1. Check manual mapping using Country_Tableau.
    country_tableau = row.get("Country_Tableau", None)
    manual = get_manual_continent(country_tableau) if country_tableau else None
    if manual:
        return manual

    # 2. Try using Country_Tableau.
    if pd.notnull(country_tableau) and country_tableau.strip():
        continent = get_continent_from_name(country_tableau.strip())
        if continent:
            return continent

    # 3. Try using ISO_A2.
    iso_a2 = row.get("ISO_A2", None)
    if pd.notnull(iso_a2) and str(iso_a2).strip():
        iso2 = str(iso_a2).strip()
        try:
            continent_code = pc.country_alpha2_to_continent_code(iso2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            if continent:
                return continent
        except Exception:
            pass

    # 4. Try using alpha-3 codes from BRK_A3, SOV_A3, ADM0_A3.
    for col in ["BRK_A3", "SOV_A3", "ADM0_A3"]:
        val = row.get(col, None)
        if pd.notnull(val) and str(val).strip():
            alpha3 = str(val).strip()
            iso2 = convert_alpha3_to_alpha2(alpha3)
            if iso2:
                try:
                    continent_code = pc.country_alpha2_to_continent_code(iso2)
                    continent = pc.convert_continent_code_to_continent_name(continent_code)
                    if continent:
                        return continent
                except Exception:
                    continue
    return None

# Updated function to determine subregion from a row.
def get_subregion_from_row(row):
    # 1. Check manual mapping using Country_Tableau.
    country_tableau = row.get("Country_Tableau", None)
    manual = get_manual_subregion(country_tableau) if country_tableau else None
    if manual:
        return manual
    
    country_name = None
    # 2. Use Country_Tableau if available.
    if pd.notnull(country_tableau) and country_tableau.strip():
        country_name = country_tableau.strip()
    
    # 3. If not, try ISO_A2.
    if not country_name:
        iso_a2 = row.get("ISO_A2", None)
        if pd.notnull(iso_a2) and str(iso_a2).strip():
            country_name = get_country_name_from_iso2(str(iso_a2).strip())
    
    # 4. If still not available, try using alpha-3 codes.
    if not country_name:
        for col in ["BRK_A3", "SOV_A3", "ADM0_A3"]:
            val = row.get(col, None)
            if pd.notnull(val) and str(val).strip():
                alpha3 = str(val).strip()
                try:
                    country = pycountry.countries.get(alpha_3=alpha3)
                    if country:
                        country_name = country.name
                        break
                except Exception:
                    continue
    
    # 5. Check manual mapping again with the resolved country name.
    if country_name:
        manual_sub = get_manual_subregion(country_name)
        if manual_sub:
            return manual_sub
    return None

# Fill missing CONTINENT values (where the field is blank or NaN)
mask_continent = df['CONTINENT'].isnull() | (df['CONTINENT'].astype(str).str.strip() == '')
df.loc[mask_continent, 'CONTINENT'] = df[mask_continent].apply(get_continent_from_row, axis=1)

# Fill missing SUBREGION values (where the field is blank or NaN)
mask_subregion = df['SUBREGION'].isnull() | (df['SUBREGION'].astype(str).str.strip() == '')
df.loc[mask_subregion, 'SUBREGION'] = df[mask_subregion].apply(get_subregion_from_row, axis=1)

In [11]:
# Export the final DataFrame to an Excel file
df.to_excel("yellow_fever_malaria_data_cleaned_yfcat_tableau_matched.xlsx", index=False)